In [1]:
import pandas as pd
from datetime import datetime
import ast
import gzip


In [2]:
with gzip.open('datasets/user_items_proc.csv.gz', 'rb') as f:
    user_items_proc = pd.read_csv(f, encoding='utf-8')
    
steam_games = pd.read_csv('./datasets/steam_games.csv', parse_dates=['release_date'])
user_review = pd.read_csv('./datasets/user_reviews.csv')

In [ ]:


diner_user = pd.merge(steam_games[['app_name','price']],user_items_proc[['user_id','items_count','item_name']],left_on='app_name',right_on='item_name',how='inner')
users_items = diner_user.groupby('user_id').agg(
    {
       'items_count':'first',
        'price':'sum'
    }
).reset_index()

recomendaciones_true = user_review.groupby('user_id')['recommend'].sum()
total_registros_por_usuario = user_review.groupby('user_id')['recommend'].count()
porcentaje_recomendaciones_true = (recomendaciones_true / total_registros_por_usuario) * 100
users_true_reviews = pd.DataFrame({
    'user_id': porcentaje_recomendaciones_true.index,
    'porcentaje_recomendaciones_true': porcentaje_recomendaciones_true.values
})
items_reviews_users = pd.merge(users_items,users_true_reviews,on='user_id',how='inner')
items_reviews_users.to_csv('./datasets/items_reviews_users.csv',index=False)

In [3]:
generos = steam_games.drop(columns=['app_name','release_date','specs','price','id','developer','Year','Accounting'])
gener_user = pd.merge(user_items_proc[['item_name','playtime_forever','user_id']],steam_games,left_on='item_name',right_on='app_name',how='inner')
resultados_por_genero = []
for genero in generos.columns:
    genero_data = gener_user[gener_user[genero] == 1]
    
    if not genero_data.empty:
        user_max = genero_data.groupby('user_id')['playtime_forever'].sum().idxmax()
        result_df = genero_data[genero_data['user_id'] == user_max].groupby('Year')['playtime_forever'].sum().reset_index()
        formatted_result = [{'Año': record['Year'], 'Horas': record['playtime_forever']} for _, record in result_df.iterrows()]
        resultados_por_genero.append({'Género': genero, 'Usuario': user_max, 'Año_Horas': formatted_result})
    else:
        resultados_por_genero.append({'Género': genero, 'Usuario': None, 'Año_Horas': []})

df_resultados = pd.DataFrame(resultados_por_genero)

df_resultados.dropna(inplace=True)

In [5]:
df_resultados.to_csv('./datasets/max_por_gen.csv',index=False)

In [25]:
def best_developer_year_func(año:str):
    func_4 = pd.merge(user_review,steam_games,left_on='item_id',right_on='id',how='inner')
    func_4 = func_4[func_4['Year'] == float(año)]
    mejores_dev = func_4.groupby('developer')['recommend'].sum().reset_index().sort_values(by='recommend',ascending=False)
    if mejores_dev.empty:
        return 'No se enocntraron reviews para items que hayan salido ese año'
    else:
        puesto1 = mejores_dev.iloc[0][0]
        puesto2 = mejores_dev.iloc[1][0]
        puesto3 = mejores_dev.iloc[2][0]
        puestos = {"Puesto 1": puesto1, "Puesto 2": puesto2, "Puesto 3": puesto3}
        return puestos

In [26]:
best_developer_year_func(2017)

{'Puesto 1': 'Smartly Dressed Games',
 'Puesto 2': 'Freejam',
 'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}

In [ ]:
user_review['posted'].iloc[0].split(',')[-1].strip(' .')

In [5]:
def developer_rec_func(desarrolladora:str):
    
    func_5 = pd.merge(user_review,steam_games,left_on='item_id',right_on='id',how='inner')
    func_5['developer'] = func_5['developer'].str.lower()
    desarrolladora = desarrolladora.lower()
    func_5 = func_5[func_5['developer'] == desarrolladora]
    if func_5.empty:
        return 'No se enocntraron reviews para items que hayan salido ese año'
    else:
        true_value = func_5[func_5['sentiment_analysis']==2]['sentiment_analysis'].count()
        false_value = func_5[func_5['sentiment_analysis']==0]['sentiment_analysis'].count()
        return {desarrolladora:[f'Nevative = {false_value}',f'Positive = {true_value}']}

In [6]:
developer_rec_func('Tripwire Interactive')

{'tripwire interactive': ['Nevative = 109', 'Positive = 319']}

In [ ]:
steam_games

In [55]:
steam_games = pd.read_csv('./datasets/steam_games.csv')
    

steam_games.dropna(subset=['Year'],inplace=True)
steam_games['Year'] = steam_games['Year'].astype(int)

In [56]:
steam_games['developer'].values

array(['Kotoshiro', 'Secret Level SRL', 'Poolians.com', ..., 'Sacada',
       'Laush Dmitriy Sergeevich', 'xropi,stev3ns'], dtype=object)

In [64]:
steam_games[steam_games['developer'].str.lower() == 'Kotoshiro'.lower()].groupby('Year')['id'].count().reset_index()

,Year,id
0,2018,1


In [74]:
steam_games[['developer','Year']]


,developer,Year
0,Kotoshiro,2018
1,Secret Level SRL,2018
2,Poolians.com,2017
3,彼岸领域,2017
4,Trickjump Games Ltd,2018
...,...,...
27549,Bidoniera Games,2018
27550,"Nikita ""Ghost_RUS""",2018
27551,Sacada,2018
27552,Laush Dmitriy Sergeevich,2018


In [70]:
def developer_func(desarrollador:str):
    
    steam_games = pd.read_csv('./datasets/steam_games.csv', parse_dates=['release_date'])
    

    steam_games.dropna(subset=['Year'],inplace=True)
    steam_games['Year'] = steam_games['Year'].astype(int)
    steam_games['developer'] = steam_games['developer'].str.strip().str.lower()
    desarrollador = desarrollador.strip().lower()   
    
    if desarrollador not in steam_games['developer'].values:
        return "No se ha encontrado ese desarrollador"  # Devuelve el mensaje si no se encuentra en el DataFrame
    
    items_por_año = steam_games[steam_games['developer'].str.lower() == desarrollador.lower()].groupby('Year')['id'].count().reset_index()
    #items_por_año['Year'] = items_por_año['Year'].astype(int)
    
    # Cuento juegos gratuitos (Free to Play) cuando 'price' es 0
    items_por_año_free = steam_games[(steam_games['developer'].str.lower() == desarrollador) & (steam_games['price'] == 0.0)].groupby('Year')['id'].count().reset_index()
    items_por_año_free.rename(columns={'id': 'Free to Play'}, inplace=True)
    
    # Uno las tablas de items_por_año e items_por_año_free
    merged_data = pd.merge(items_por_año, items_por_año_free, on='Year', how='left')
    
    # Calculo el contenido gratis por año
    contenido_free = round(merged_data['Free to Play'] / merged_data['id'] * 100, 2)
    
    # Creo el DataFrame final
    resultado = {
        'Año': merged_data['Year'].tolist(),
        'Cantidad de Items': merged_data['id'].tolist(),
        'Contenido Free': f'{contenido_free.fillna(0).tolist()} %'  # Llenar NaN con 0 para evitar problemas
    }
    
    return resultado

In [77]:
developer_func('Dovetail Games')

{'Año': [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
 'Cantidad de Items': [22, 4, 4, 52, 39, 37, 50, 21, 24],
 'Contenido Free': '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.17] %'}